# Load libraries

In [3]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
from torchvision import datasets, transforms, models
import pathlib

# Checking for device 

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Data loading

In [5]:
data_transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

In [6]:
train_path='./seg_train/seg_train'
test_path='./seg_test/seg_test'

train_loader= DataLoader(datasets.ImageFolder(train_path,transform=data_transformer), batch_size=64, shuffle=True)
test_loader= DataLoader(datasets.ImageFolder(test_path,transform=data_transformer), batch_size=32, shuffle=True)

In [7]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


# CNN Network

In [9]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=6):
        super(ConvNet, self).__init__()

        # Output size after convolution filter
        # ((w - f + 2P) / s) + 1
        
        # Inpit shape = (256, 3, 150, 150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # Shape (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape (256, 12, 150, 150)
        self.relu1 = nn.ReLU()
        # Shape (256, 12, 150, 150)

        # Reudce the image size by factor of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        # Shape (256, 12, 75, 75)

        self.conv1 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        # Shape (256, 20, 75, 75)
        self.relu2 = nn.ReLU()
        # Shape (256, 20, 75, 75)

        self.conv1 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Shape (256, 32, 75, 75)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape (256, 32, 75, 75)
        self.relu1 = nn.ReLU()
        # Shape (256, 32, 75, 75)

        self.fc = nn.Linear(in_features=32*75*75, out_features=num_classes)

        # Feed forward function 
        def forward(self,input):
            output=self.conv1(input)
            output=self.bn1(output)
            output=self.relu1(output)
                
            output=self.pool(output)
                
            output=self.conv2(output)
            output=self.relu2(output)
                
            output=self.conv3(output)
            output=self.bn3(output)
            output=self.relu3(output)
                    
            #Above output will be in matrix form, with shape (256,32,75,75)
                
            output=output.view(-1,32*75*75)
                
            output=self.fc(output)
                
            return output

In [ ]:
model = ConvNet(num_classes=6).to(device)